<div style="text-align: center;">
    <h1> <strong>Nom et Prénom : Laakel Gauzi Soumaya</strong> </h1>
    <h3><strong>Module : NATURAL LANGUAGE PROCESSING (NLP)</strong></h3>
    <h4><em>MST IASD/S1 2024-2025</em></h4>
    <hr>
    <h2><strong>DEVOIR 5 : ANALYSE SYNTAXIQUE DE COMPTES-RENDUS MÉDICAUX</strong></h2>
    <p><i>L’objectif du Devoir 5 est de développer un système d’analyse automatique de comptes-rendus médicaux non structurés, dans un contexte hospitalier. Ce devoir vise à extraire des relations pertinentes entre les patients, les symptômes et les traitements à partir de textes cliniques rédigés en langage naturel. Pour cela, il s’appuie sur des techniques de traitement automatique du langage naturel (TALN), notamment le POS tagging, l’analyse des dépendances grammaticales, l’identification des relations sujet-verbe-objet, ainsi que la détection des négations. L’enjeu principal est de structurer l’information médicale afin de faciliter son exploitation pour des analyses ultérieures, des aides au diagnostic, ou des applications en santé numérique.</i></p>
</div>

# Partie 1 : POS Tagging Médical

## Étape 1 : Chargement du modèle et du texte

In [1]:
import spacy

# charge le modèle français de spaCy
nlp = spacy.load("fr_core_news_sm")

# le texte médical
texte = "La patiente âgée de 65 ans présente une toux persistante et une fièvre à 38.5°C. Le médecin prescrit de l'amoxicilline 500mg 3x/jour."
doc = nlp(texte)

## Étape 2 : Affichage des tokens avec POS, lemme, dépendance

In [2]:
# Je parcours chaque mot et j'affiche les informations importantes
for token in doc:
    print(f"{token.text:<15} | POS: {token.pos_:<10} | Lemma: {token.lemma_:<15} | Dépendance: {token.dep_}")

La              | POS: DET        | Lemma: le              | Dépendance: det
patiente        | POS: NOUN       | Lemma: patient         | Dépendance: nsubj
âgée            | POS: VERB       | Lemma: âger            | Dépendance: acl
de              | POS: ADP        | Lemma: de              | Dépendance: case
65              | POS: NUM        | Lemma: 65              | Dépendance: nummod
ans             | POS: NOUN       | Lemma: an              | Dépendance: obl:arg
présente        | POS: VERB       | Lemma: présente        | Dépendance: ROOT
une             | POS: DET        | Lemma: un              | Dépendance: det
toux            | POS: ADP        | Lemma: toux            | Dépendance: case
persistante     | POS: NOUN       | Lemma: persistante     | Dépendance: obj
et              | POS: CCONJ      | Lemma: et              | Dépendance: cc
une             | POS: DET        | Lemma: un              | Dépendance: det
fièvre          | POS: NOUN       | Lemma: fièvre          | Dépe

### **Analyse phrase par phrase :**

#### **1.** *"La patiente âgée de 65 ans présente une toux persistante et une fièvre à 38.5°C."*

* **La** (DET) est un **déterminant** qui détermine **patiente**.
* **patiente** (NOUN) est le **sujet grammatical** (`nsubj`) du verbe **présente**.
* **âgée** (VERB, lemma "âger", mais interprété comme adjectif verbal) exprime une caractéristique de la patiente — c’est une **subordonnée relative ou un complément du nom** (`acl` pour *adjectival clause*).
* **de 65 ans** est une construction prépositionnelle :

  * **de** introduit la relation,
  * **65** (NUM) est le **modificateur numérique** de **ans**,
  * **ans** (NOUN) est un **complément circonstanciel de temps ou d'âge** (`obl:arg`).
* **présente** est le **verbe principal** de la phrase, identifié comme **racine** syntaxique (`ROOT`).
* **une toux persistante et une fièvre à 38.5°C** est le **complément d’objet direct** (`obj`) du verbe :

  * **toux** est erronément tagué comme ADP, mais devrait être un nom.
  * **persistante** est le qualificatif (adjectif) associé à **toux**.
  * **fièvre** est en coordination avec **toux** (`conj`) introduite par **et** (`cc`).
  * **à 38.5°C** est un **complément de mesure** attaché à **fièvre**, avec **nmod** (modificateur nominal).

---

#### **2.** *"Le médecin prescrit de l'amoxicilline 500 mg 3x/jour."*

* **Le médecin** (DET + NOUN) est le **sujet** (`nsubj`).
* **prescrit** est analysé comme un ADJ à tort (probable erreur), mais devrait être un **verbe racine** (`ROOT`) signifiant l’action principale.
* **de l'amoxicilline** est un **complément introduit par préposition** (`obl:arg`) :

  * **de** est la préposition,
  * **l'** est le déterminant,
  * **amoxicilline** est le **nom du médicament**.
* **500 mg** est la **posologie** :

  * **500** (NUM) est le **modificateur** de **mg**,
  * **mg** (milligrammes) est un **objet direct** (`obj`) lié à l’action de prescrire.
* **3x/jour** est un **complément de fréquence** ou de dosage :

  * **3x** est l’adjectif modifiant la fréquence,
  * **jour** est le **nom modifié** (`nmod`), et **/** est traité comme un séparateur symbolique.

---

### **Problèmes Identifiés :**

* L’analyse contient **quelques erreurs de POS tagging**, notamment :

  * "toux" → tagué comme ADP (préposition) au lieu de NOUN.
  * "prescrit" → identifié comme ADJ au lieu de VERB.
  * Tokenisation de "38.5°C" :
    - Séparé en 38.5, °, C → devrait être traité comme une entité unique (température).
  * Tokenisation de "3x/jour" :
    - Séparé en 3x, /, jour → devrait être une entité unique (posologie). 

## Étape 3 : Corriger la tokenisation

In [3]:
import spacy
from spacy.tokens import Span

# Afficher les tokens initiaux
print("Avant correction :")
print([token.text for token in doc])

# Retokenization
with doc.retokenize() as retokenizer:
    # Fusionner "38.5", "°", "C" s'ils existent
    for i in range(len(doc) - 2):
        if doc[i].text == "38.5" and doc[i+1].text == "°" and doc[i+2].text.lower() == "c":
            span_temp = doc[i:i+3]
            retokenizer.merge(span_temp, attrs={"ent_type": "TEMPÉRATURE"})

    # Fusionner "3x", "/", "jour"
    for i in range(len(doc) - 2):
        if doc[i].text == "3x" and doc[i+1].text == "/" and doc[i+2].text.lower() == "jour":
            span_poso = doc[i:i+3]
            retokenizer.merge(span_poso, attrs={"ent_type": "POSOLOGIE"})

    # Fusionner les doses comme "500" + "mg"
    for i in range(len(doc) - 1):
        if doc[i].like_num and doc[i+1].text.lower() in ["mg", "g", "ml"]:
            span_dose = doc[i:i+2]
            retokenizer.merge(span_dose, attrs={"ent_type": "DOSE"})

# Afficher les tokens après correction
print("\nAprès correction :")
print([token.text for token in doc])

# Afficher les types d'entités (si définis)
for token in doc:
    print(f"{token.text} -> ent_type: {token.ent_type_}")


Avant correction :
['La', 'patiente', 'âgée', 'de', '65', 'ans', 'présente', 'une', 'toux', 'persistante', 'et', 'une', 'fièvre', 'à', '38.5', '°', 'C', '.', 'Le', 'médecin', 'prescrit', 'de', "l'", 'amoxicilline', '500', 'mg', '3x', '/', 'jour', '.']

Après correction :
['La', 'patiente', 'âgée', 'de', '65', 'ans', 'présente', 'une', 'toux', 'persistante', 'et', 'une', 'fièvre', 'à', '38.5°C', '.', 'Le', 'médecin', 'prescrit', 'de', "l'", 'amoxicilline', '500mg', '3x/jour', '.']
La -> ent_type: 
patiente -> ent_type: 
âgée -> ent_type: 
de -> ent_type: 
65 -> ent_type: 
ans -> ent_type: 
présente -> ent_type: 
une -> ent_type: 
toux -> ent_type: 
persistante -> ent_type: 
et -> ent_type: 
une -> ent_type: 
fièvre -> ent_type: 
à -> ent_type: 
38.5°C -> ent_type: TEMPÉRATURE
. -> ent_type: 
Le -> ent_type: 
médecin -> ent_type: 
prescrit -> ent_type: 
de -> ent_type: 
l' -> ent_type: 
amoxicilline -> ent_type: 
500mg -> ent_type: DOSE
3x/jour -> ent_type: POSOLOGIE
. -> ent_type: 


In [4]:
# Je parcours chaque mot et j'affiche les informations importantes
for token in doc:
    print(f"{token.text:<15} | POS: {token.pos_:<10} | Lemma: {token.lemma_:<15} | Dépendance: {token.dep_}")

La              | POS: DET        | Lemma: le              | Dépendance: det
patiente        | POS: NOUN       | Lemma: patient         | Dépendance: nsubj
âgée            | POS: VERB       | Lemma: âger            | Dépendance: acl
de              | POS: ADP        | Lemma: de              | Dépendance: case
65              | POS: NUM        | Lemma: 65              | Dépendance: nummod
ans             | POS: NOUN       | Lemma: an              | Dépendance: obl:arg
présente        | POS: VERB       | Lemma: présente        | Dépendance: ROOT
une             | POS: DET        | Lemma: un              | Dépendance: det
toux            | POS: ADP        | Lemma: toux            | Dépendance: case
persistante     | POS: NOUN       | Lemma: persistante     | Dépendance: obj
et              | POS: CCONJ      | Lemma: et              | Dépendance: cc
une             | POS: DET        | Lemma: un              | Dépendance: det
fièvre          | POS: NOUN       | Lemma: fièvre          | Dépe

## Étape 4 : Filtrage par Catégories

### 4.1 Symptômes (noms + adjectifs) :

In [5]:
symptomes = []
for chunk in doc.noun_chunks:
    if "toux" in chunk.text or "fièvre" in chunk.text:  # Adapter avec des termes médicaux
        symptomes.append(chunk.text)
print("Symptômes :", symptomes)

Symptômes : ['une toux persistante', 'une fièvre']


### 4.2 Traitements (verbes + médicaments) :

In [6]:
traitements = []
for token in doc:
    # Identifier le verbe "prescrire" (lemme) et vérifier qu'il est racine
    if token.lemma_ == "prescrire" and token.dep_ == "ROOT":
        # Parcourir ses enfants (objets, compléments obliques)
        for child in token.children:
            # Inclure obj, nmod, et tous les sous-types de obl (ex: obl:arg)
            if child.dep_ in ("obj", "nmod") or child.dep_.startswith("obl"):
                # Extraire le sous-arbre complet du complément (ex: "amoxicilline 500mg 3x/jour")
                traitement_span = child.subtree
                # Convertir en texte
                traitement = " ".join([t.text for t in traitement_span])
                traitements.append(traitement)

print("Traitements :", traitements)

Traitements : ["de l' amoxicilline", '500mg 3x/jour']


# Partie 2 : Extraction de Relations

## Étape 1 : Fonction pour extraire Sujet-Verbe-Objet

In [7]:
def extraire_relations(phrase):
    doc = nlp(phrase)
    relations = []

    for token in doc:
        # Cas 1 : Sujet explicite (nsubj) + verbe + objet
        if token.dep_ in ("nsubj", "expl:subj") and token.head.pos_ == "VERB":
            verbe = token.head
            # Recherche de l'objet direct ou indirect du verbe
            obj_direct = [child for child in verbe.children if child.dep_ in ("obj", "obl:arg", "xcomp")]
            
            # Si le verbe a un complément clausal (ex: "faut prescrire")
            for child in verbe.children:
                if child.dep_ == "xcomp":
                    # Recherche de l'objet du complément clausal
                    obj_xcomp = [c for c in child.children if c.dep_ in ("obj", "obl:arg")]
                    if obj_xcomp:
                        relations.append((token.text, verbe.text, obj_xcomp[0].text))
            
            # Ajout de l'objet direct du verbe principal
            if obj_direct:
                relations.append((token.text, verbe.text, obj_direct[0].text))

        # 2. Cause : "en raison de X" → ("X", "raison", verbe principal)
        if token.text.lower() == "raison":
            # Vérifie si le token précédent est "en"
            idx = token.i
            if idx > 0 and doc[idx - 1].text.lower() == "en":
                # Cherche le mot après "de"
                for child in token.children:
                    if child.dep_ == "nmod" or child.dep_ == "obl":
                        cause = child.text
                        # le verbe est l'ancêtre le plus haut dans la phrase
                        verb = token.head
                        while verb.head != verb:
                            verb = verb.head
                        relations.append((cause, "raison", verb.text))

    return relations

In [8]:
phrase = "Le médecin arrête l'aspirine en raison de saignements."
print(extraire_relations(phrase))

[('médecin', 'arrête', 'aspirine'), ('saignements', 'raison', 'arrête')]


## Étape 2 : Gestion des négations

In [9]:
def extraire_relations_neg(phrase):
    doc = nlp(phrase)
    relations = []

    # Cas 1 : Structure avec verbe principal + négation
    for token in doc:
        if token.dep_ in ("obj", "obl") and token.head.pos_ == "VERB":
            # Trouver le sujet
            sujet = [w for w in token.head.lefts if w.dep_ in ("nsubj", "expl:subj")]
            # Détecter les négations
            negations = [w for w in token.head.children if w.dep_ == "neg" or w.text.lower() in ("ne", "pas")]
            
            if sujet and negations:
                verbe_form = f"NE PAS {token.head.lemma_}"
                relations.append((sujet[0].text, verbe_form, token.text))

    # Cas 2 : Structure "falloir" + infinitif + négation
    for token in doc:
        if token.lemma_ == "falloir":
            # Vérifier la négation au niveau de "falloir"
            neg_falloir = any(w.text.lower() in ("ne", "pas") for w in token.head.children)
            
            # Chercher les verbes à l'infinitif dépendants de "falloir"
            for child in token.children:
                if child.dep_ == "xcomp" and child.pos_ == "VERB":
                    # Chercher l'objet de l'infinitif
                    for obj in child.children:
                        if obj.dep_ in ("obj", "obl"):
                            if neg_falloir:
                                relations.append(("—", f"NE PAS {child.lemma_}", obj.text))

    # Cas 3 : Forme "Pas de X" (négation sans verbe explicite)
    for token in doc:
        if token.text.lower() == "pas":
            for child in token.children:
                if child.dep_ in ("obj", "nmod"):
                    relations.append(("—", "NE PAS utiliser", child.text))

    return relations

In [10]:
print(extraire_relations_neg("Le médecin ne prescrit pas de paracétamol."))

[('médecin', 'NE PAS prescrire', 'paracétamol')]


# Partie 3 : Analyse de Dépendances

### Étape 1 : Visualisation avec displaCy

In [11]:
from spacy import displacy

#visualise les dépendances
texte_visu = "Après analyse, le cardiologue recommande un scanner cardiaque immédiat."
doc_visu = nlp(texte_visu)
displacy.render(doc_visu, style='dep', jupyter=True)

## Étape 2 : Requête Médicament + Posologie

In [12]:
import spacy
import re

nlp = spacy.load("fr_core_news_sm")

def est_posologie(token):
    texte = token.text.lower()
    return (
        token.like_num or
        "mg" in texte or
        "g" in texte or
        "ml" in texte or
        texte in {"x", "fois", "comprimé", "comprimés", "jour", "semaine", "matin", "soir"} or
        re.match(r"\d+x", texte) or  # Capture "3x"
        token.is_punct  # Capture "/" dans "3x/jour"
    )0

def extraire_posologie(doc):
    relations = []
    i = 0
    while i < len(doc):
        token = doc[i]
        # Recherche des médicaments (noms communs, adapté aux cas comme "amoxicilline")
        if token.pos_ == "NOUN" and token.text.islower():
            med = token.text
            posologie_parts = []
            j = i + 1
            
            # Parcourir les tokens suivants pour former la posologie
            while j < len(doc) and (est_posologie(doc[j]) or doc[j].text == "/"):
                current_token = doc[j]
                # Fusionner "500" + "mg" → "500mg"
                if current_token.like_num and j + 1 < len(doc) and doc[j+1].text in {"mg", "g", "ml"}:
                    posologie_parts.append(f"{current_token.text}{doc[j+1].text}")
                    j += 2  # Sauter le token suivant déjà traité
                # Fusionner "3x" + "/" + "jour" → "3x/jour"
                elif re.match(r"\d+x", current_token.text) and j + 2 < len(doc) and doc[j+1].text == "/":
                    posologie_parts.append(f"{current_token.text}/{doc[j+2].text}")
                    j += 3
                else:
                    posologie_parts.append(current_token.text)
                    j += 1
            
            if posologie_parts:
                relations.append((med, " ".join(posologie_parts)))
            i = j  # Mettre à jour l'index
        else:
            i += 1
    return relations

In [13]:
# Test avec la phrase
texte = "Le médecin prescrit de l'amoxicilline 500 mg 3x / jour."
doc = nlp(texte)
print(extraire_posologie(doc)) 

[('amoxicilline', '500mg 3x/jour .')]


# Analyse sur 5 phrases test sur Extraction de relations et de Négations. 

In [14]:
phrase1 = "Le patient ne prend pas ses médicaments."
print(extraire_relations(phrase1))

[('patient', 'prend', 'médicaments')]


In [15]:
print(extraire_relations_neg(phrase1))

[('patient', 'NE PAS prendre', 'médicaments')]


In [16]:
phrase2 ="En raison de la fièvre, l'infirmière administre du paracétamol."
print(extraire_relations(phrase2))

[('fièvre', 'raison', 'administre'), ('infirmière', 'administre', 'paracétamol')]


In [17]:
extraire_relations_neg(phrase2)

[]

In [18]:
phrase3 ="Le cardiologue recommande un scanner cardiaque immédiat."
print(extraire_relations(phrase3))

[('cardiologue', 'recommande', 'scanner')]


In [19]:
extraire_relations_neg(phrase3)

[]

In [20]:
phrase4 = "Pas de traitement antibiotique pour cette infection."
print(extraire_relations(phrase4))

[]


In [21]:
extraire_relations_neg(phrase4)

[('—', 'NE PAS utiliser', 'traitement')]

In [22]:
phrase5 ="Il ne faut pas prescrire le paracétamol dans ce cas."
print(extraire_relations(phrase5))

[('Il', 'faut', 'prescrire')]


In [23]:
extraire_relations_neg(phrase5)

[('Il', 'NE PAS falloir', 'paracétamol')]

In [24]:
phrases_test = [
        "Le patient refuse l'anticoagulant malgré son AVC récent.", 
        "Prescription : ibuprofène 400mg si douleur, maximum 3 comprimés/jour.", 
        "Pas d'antibiothérapie pour cette infection virale." ,
        "Le patient ne prend pas ses médicaments.",
        "En raison de la fièvre, l'infirmière administre du paracétamol.",
        "Le cardiologue recommande un scanner cardiaque immédiat.",
        "Pas de traitement antibiotique pour cette infection.",
        "Il ne faut pas prescrire le paracétamol dans ce cas."
    ]

for phrase in phrases_test:
    print("\nPhrase :", phrase)
    print("Relations positives :", extraire_relations(phrase))
    print("Relations avec négation :", extraire_relations_neg(phrase))


Phrase : Le patient refuse l'anticoagulant malgré son AVC récent.
Relations positives : [('patient', 'refuse', 'anticoagulant')]
Relations avec négation : []

Phrase : Prescription : ibuprofène 400mg si douleur, maximum 3 comprimés/jour.
Relations positives : []
Relations avec négation : []

Phrase : Pas d'antibiothérapie pour cette infection virale.
Relations positives : []
Relations avec négation : [('—', 'NE PAS utiliser', 'antibiothérapie')]

Phrase : Le patient ne prend pas ses médicaments.
Relations positives : [('patient', 'prend', 'médicaments')]
Relations avec négation : [('patient', 'NE PAS prendre', 'médicaments')]

Phrase : En raison de la fièvre, l'infirmière administre du paracétamol.
Relations positives : [('fièvre', 'raison', 'administre'), ('infirmière', 'administre', 'paracétamol')]
Relations avec négation : []

Phrase : Le cardiologue recommande un scanner cardiaque immédiat.
Relations positives : [('cardiologue', 'recommande', 'scanner')]
Relations avec négation : 